In [1]:
import librosa
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from librosa import display
import os, re, json
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score, precision_score

In [3]:
import sys
sys.path.append("./VGGish")
sys.path.append("./tmp")

import vggish_slim
import vggish_params
import vggish_input
from load_data import ToiletData

# Audio and VGGish

## Find audio files

In [5]:
data_capture_id = 989
data_frames_dir = "data/data_frames/"
for subdir, dirs, files in os.walk(data_frames_dir):
    if subdir.split("_")[-1] == str(data_capture_id) and \
        'back_audio_data.wav' in files:
        print (os.path.join(subdir, 'back_audio_data.wav'))

data/data_frames/data_capture_989\back_audio_data.wav


In [6]:
parent = os.getcwd()
data_frames_dir = os.path.join(parent, "data", "data_frames")
print ("Searching audio files in {}".format(data_frames_dir))
print ("-"*50)

audio_info_df = []
for subdir, dirs, files in os.walk(data_frames_dir):
    for file in files:
        if file.endswith('.wav'):
            data_capture_id = subdir.split('\\')[-1]
            data_capture_id = int(''.join([i for i in data_capture_id if i.isdigit()]))
            filepath = os.path.join(subdir, file)
            audio_info_df.append([filepath, data_capture_id, file])

audio_info_df = pd.DataFrame(audio_info_df)
audio_info_df.columns = ['PATH', 'Index', 'File']

print ("Find {} audio files".format(audio_info_df.shape[0]))
print ()
print ("File summary")
print (audio_info_df.File.value_counts())

Searching audio files in C:\Users\Jiajun\Desktop\20Spring\VGGishAudio\data\data_frames
--------------------------------------------------
Find 817 audio files

File summary
back_audio_data.wav     378
front_audio_data.wav    378
audio_data.wav           61
Name: File, dtype: int64


In [7]:
# we only need back_audio_file and user index between 953 and 1000 for now
# extract them from the audio_info_df

backaudio_info_df = audio_info_df[(audio_info_df.File == "back_audio_data.wav") \
                                 & (audio_info_df.Index < 996) & (audio_info_df.Index > 950)]
backaudio_info_df.head()

,PATH,Index,File
729,C:\Users\Jiajun\Desktop\20Spring\VGGishAudio\d...,952,back_audio_data.wav
731,C:\Users\Jiajun\Desktop\20Spring\VGGishAudio\d...,953,back_audio_data.wav
733,C:\Users\Jiajun\Desktop\20Spring\VGGishAudio\d...,954,back_audio_data.wav
735,C:\Users\Jiajun\Desktop\20Spring\VGGishAudio\d...,955,back_audio_data.wav
737,C:\Users\Jiajun\Desktop\20Spring\VGGishAudio\d...,956,back_audio_data.wav


In [9]:
# label
with open("tmp/classifier.json") as json_data_file:
    config = json.load(json_data_file)

Annotations = ToiletData(config).get_annotations('annotation.csv')

In [10]:
sr = 96000
data_captures = [i for i in backaudio_info_df.Index.values if i in Annotations.keys()]

In [12]:
print (data_captures)

[952, 953, 954, 955, 956, 957, 958, 959, 960, 963, 964, 965, 967, 968, 969, 970, 971, 972, 974, 976, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 989, 990, 992, 993, 994, 995]


## Neural network architecture

In [ ]:
tf.reset_default_graph()
sess = tf.Session()
vgg = CreateVGGishNetwork()

class Input_Audio:
    def __init__(self, data_captures):
        self.data_captures = data_captures
        self.sr = 96000
        self.window_size = 1
    
    
    def _get_windowed_samples(self, data_capture_id):
        path = backaudio_info_df.PATH[backaudio_info_df.Index == data_capture_id]
        assert len(path.values) == 1
        
        path = path.values[0]
        x, sampling_rate = librosa.load(path, self.sr)
        window_samples = self.window_size * self.sr
        time_steps = x.shape[0] // window_samples               # number of time_steps
        x = x[:time_steps * window_samples]                     # make even splices
        windowed_x = np.array(np.split(x, time_steps))
        
        return windowed_x
    
    
    def _get_embedding(self, data_capture_id):
        res = []
        windowed_x = self._get_windowed_samples(data_capture_id)
        for sample in windowed_x:
            res.append(EmbeddingsFromVGGish(vgg, sample, self.sr).flatten())
        res = np.array(res)[np.newaxis, :]
        
        return res      # (1, time_steps, 128)
    
    
    def _get_label(self, data_capture_id):
        path = backaudio_info_df.PATH[backaudio_info_df.Index == data_capture_id]
        assert len(path.values) == 1
        
        path = path.values[0]
        x, sampling_rate = librosa.load(path, self.sr)
        window_samples = self.window_size * self.sr
        time_steps = x.shape[0] // window_samples
        
        label = np.zeros(time_steps)
        for annotation in Annotations[data_capture_id]:
            if annotation[-1] == 'Urination':
                start, end = map(lambda x: int(x)/1e6, annotation[:2])
                label[int(start):int(end)+1] = 1
        
        return np.array(label)
        
    
    def _get_all_embeddings(self):
        res = []
        for data_capture_id in self.data_captures:
            print ('updating embedding for {}'.format(data_capture_id))
            res.append(self._get_embedding(data_capture_id))
        
        return res       # list: (num_of_examples, )
    
    
    def _get_all_labels(self):
        res = []
        for data_capture_id in self.data_captures:
            print ('updating label for {}'.format(data_capture_id))
            res.append(self._get_label(data_capture_id))
        
        return res       # list: (num_of_examples, )

In [ ]:
ins = Input_Audio(data_captures)
y = ins._get_all_labels()
x = ins._get_all_embeddings()

In [ ]:
tf.keras.backend.clear_session()
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True, input_shape=[None, 128]),
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=[None, 32]),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(2, activation="softmax"))
])

model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.01),
             metrics=['accuracy'])

## Training and testing

In [ ]:
train_ind = np.random.choice(len(x), int(0.7*len(x)), replace=False)
dev_ind = [i for i in range(len(x)) if i not in train_ind]

train_X = [x[i] for i in train_ind]
dev_X = [x[i] for i in dev_ind]
train_y = [y[i] for i in train_ind]
dev_y = [y[i] for i in dev_ind]

def train_generator():
    ind = 0
    while True:
        xbatch = train_X[ind % len(train_X)]
        ybatch = tf.keras.utils.to_categorical(train_y[ind % len(train_y)])[np.newaxis, :]
        ind += 1
        yield xbatch, ybatch

def validation_generator():
    ind = 0
    while True:
        xbatch = dev_X[ind % len(dev_X)]
        ybatch = tf.keras.utils.to_categorical(dev_y[ind % len(dev_y)])[np.newaxis, :]
        ind += 1
        yield xbatch, ybatch

print ('Training example: {}'.format(len(train_X)))
print ('Validation example: {}'.format(len(dev_X)))

In [ ]:
model.fit_generator(train_generator(), validation_data=validation_generator(),
                   steps_per_epoch=len(train_X) // 1, validation_steps = len(dev_X) // 1,
                   epochs=15, verbose=2)

In [ ]:
# evaluation on dev set single use case
threshold = 0.3
accuracy, recall, precision = [], [], []
for i in range(len(dev_ind)):
    xbatch, ybatch = dev_X[i], dev_y[i]
    ypred_batch = (model.predict(xbatch).reshape(-1, 2)[:, 1] > threshold).astype(int)
    accuracy.append(np.sum(ypred_batch == ybatch) / ybatch.shape[0])
    recall.append(recall_score(ybatch, ypred_batch))
    precision.append(precision_score(ybatch, ypred_batch))
    #print (np.sum(ypred_batch == ybatch) / ybatch.shape[0])

In [ ]:
print ([data_captures[i] for i in train_ind])

In [ ]:
# evaluation on dev set overall
threshold = 0.5
preds, labels = [], []
for i in range(len(dev_ind)):
    xbatch, ybatch = dev_X[i], dev_y[i]
    preds += list((model.predict(xbatch).reshape(-1, 2)[:, 1] > threshold))
    labels += list(ybatch)

assert len(preds) == len(labels)

print (classification_report(labels, preds))

In [ ]:
# evaluation on dev set overall
threshold = 0.7
preds, labels = [], []
for i in range(len(dev_ind)):
    xbatch, ybatch = dev_X[i], dev_y[i]
    preds += list((model.predict(xbatch).reshape(-1, 2)[:, 1] > threshold))
    labels += list(ybatch)

assert len(preds) == len(labels)

print (classification_report(labels, preds))

In [ ]:
# evaluation on dev set overall
threshold = 0.9
preds, labels = [], []
for i in range(len(dev_ind)):
    xbatch, ybatch = dev_X[i], dev_y[i]
    preds += list((model.predict(xbatch).reshape(-1, 2)[:, 1] > threshold))
    labels += list(ybatch)

assert len(preds) == len(labels)

print (classification_report(labels, preds))

In [ ]:
fig, axes = plt.subplots(1, 3)

axes[0].boxplot(np.array(accuracy))
axes[1].boxplot(np.array(recall))
axes[2].boxplot(np.array(precision))
pass

In [ ]:
clf_eval = pd.DataFrame(np.array((accuracy, recall, precision)).transpose())
clf_eval.index = [data_captures[i] for i in dev_ind]
clf_eval.columns = ["Accuracy", "Recall", "Precision"]
clf_eval

## Urination Evaluation Plots

In [ ]:
def find_regions(boolean_list):
    #boolean_list = y[data_captures.index(data_capture_id)]
    a_list = [i for i, x in enumerate(boolean_list) if x]
    res = []
    start = 0
    i = 0
    while i < len(a_list) - 1:
        if a_list[i+1] - a_list[i] == 1:
            i += 1
        else:
            res.append([a_list[start], a_list[i]])
            start = i + 1
            i += 1
    
    if i < len(a_list):
        res.append([a_list[start], a_list[i]])
    
    return res

In [ ]:
dev_data_capture_ids = [data_captures[i] for i in dev_ind]
dev_data_capture_ids

In [ ]:
def GetAudio(use_i):
    sampleRate_n = 44100
    wav_fn  = 'May17/data/data_frames/data_capture_{}/back_audio_data.wav'.format(use_i)
    x, fs = librosa.load(wav_fn,sr=sampleRate_n)

    n_fft = 2048
    hop_length = 512
    n_mels = 128
    S = librosa.feature.melspectrogram(x, sr=sampleRate_n, n_fft=n_fft, 
                                       hop_length=hop_length, 
                                       n_mels=n_mels)
    S_DB = librosa.power_to_db(S, ref=np.max)
    return S_DB, fs, hop_length

In [ ]:
def plot_eval(data_capture_id):
    # get total weight and radar energy
    total_weight = ToiletData(config).get_total_weight_sz(data_capture_id)
    total_weight.index = (total_weight.index - total_weight.index[0]) / 1e6
    S_DB, fs, hop_len = GetAudio(data_capture_id)
    
    fig, ax = plt.subplots(3, 1, figsize = (12, 6), sharex = True)
    ax[0].plot(total_weight)
    ax[0].grid()
    ax[1].plot(total_weight)
    ax[1].grid()
    ax[2] = librosa.display.specshow(S_DB, sr=fs, hop_length=hop_len, x_axis='time', y_axis='mel')
    
    
    annotated_list = y[data_captures.index(data_capture_id)]
    predicted_list = model.predict(x[data_captures.index(data_capture_id)]).reshape(-1, 2)[:, 1] > threshold
    
    for region in find_regions(annotated_list):
        ax[0].axvspan(region[0], region[1]+1, alpha=0.5, color='red')
    
    for region in find_regions(predicted_list):
        ax[1].axvspan(region[0], region[1]+1, alpha=0.5, color='orange')
    
    ax[0].set_title("Annotated: {}".format(data_capture_id))
    ax[0].set_ylim(total_weight.median()-1, total_weight.median()+1)
    ax[1].set_title("Predicted: {}".format(data_capture_id))
    ax[1].set_ylim(total_weight.median()-1, total_weight.median()+1)
    
    ax[2].xaxis.set_major_locator(plt.MaxNLocator(30))
    
    fig.tight_layout(pad=0.5)
    plt.savefig("eval_urine_{}".format(ind))
    #plt.show()

In [ ]:
for ind in dev_data_capture_ids:
    plot_eval(ind)
    #plt.savefig("eval_{}".format(ind))

## Helper functions

In [ ]:
# vggish helper
def CreateVGGishNetwork(hop_size=0.96):   # Hop size is in seconds.
    vggish_slim.define_vggish_slim()
    checkpoint_path = './VGGish/vggish_model.ckpt'
    vggish_params.EXAMPLE_HOP_SECONDS = hop_size
  
    vggish_slim.load_vggish_slim_checkpoint(sess, checkpoint_path)

    features_tensor = sess.graph.get_tensor_by_name(
        vggish_params.INPUT_TENSOR_NAME)
    embedding_tensor = sess.graph.get_tensor_by_name(
        vggish_params.OUTPUT_TENSOR_NAME)

    layers = {'conv1': 'vggish/conv1/Relu',
              'pool1': 'vggish/pool1/MaxPool',
              'conv2': 'vggish/conv2/Relu',
              'pool2': 'vggish/pool2/MaxPool',
              'conv3': 'vggish/conv3/conv3_2/Relu',
              'pool3': 'vggish/pool3/MaxPool',
              'conv4': 'vggish/conv4/conv4_2/Relu',
              'pool4': 'vggish/pool4/MaxPool',
              'fc1': 'vggish/fc1/fc1_2/Relu',
              'fc2': 'vggish/fc2/Relu',
              'embedding': 'vggish/embedding',
              'features': 'vggish/input_features',
             }
    g = tf.get_default_graph()
    for k in layers:
        layers[k] = g.get_tensor_by_name( layers[k] + ':0')
    
    return {'features': features_tensor,
            'embedding': embedding_tensor,
            'layers': layers,
           }


def EmbeddingsFromVGGish(vgg, x, sr):
    # Produce a batch of log mel spectrogram examples.
    input_batch = vggish_input.waveform_to_examples(x, sr)
    # print('Log Mel Spectrogram example: ', input_batch[0])
    
    layer_names = vgg['layers'].keys()
    tensors = [vgg['layers'][k] for k in layer_names]
    
    results = sess.run(tensors,
                       feed_dict={vgg['features']: input_batch})
    resdict = {}
    for i, k in enumerate(layer_names):
        resdict[k] = results[i]
    
    return resdict['embedding']

In [ ]:
#from tf.keras.utils import to_categorical
def train_generator():
    j = 0
    while j <= 5:
        j += 1
        sequence_length = np.random.randint(10, 100)
        x_train = np.random.random((1000, sequence_length, 5))
        # y_train will depend on past 5 timesteps of x
        y_train = x_train[:, :, 0]
        for i in range(1, 5):
            y_train[:, i:] += x_train[:, :-i, i]
        y_train = tf.keras.utils.to_categorical(y_train > 2.5)
        print (x_train.shape, y_train.shape)